In [218]:
import numpy as np

def bingo_input(input_file:str):
    """Parse input bingo data - first two lines are called numbers, rest are bingo board arrays"""
    depths = []    
    with open(input_file, 'r') as file:
        # Read first line as the called numbers into list
        callnums = file.readline().strip().split(',')
        callnums = list(map(int, callnums))
        
        # Now create list of arrays for the boards
        bingo_boards = []
        current_board = []
        for line in file:
            if len(line) <= 1:
                if len(current_board) > 1:
                    bingo_boards.append(np.array(current_board).astype(int))
                current_board = []
            else:
                current_board.append(line.strip().split())
        # Catch last line
        bingo_boards.append(np.array(current_board).astype(int))

    return callnums, bingo_boards

In [503]:
class Bingo(object):
    """ 
    Contains bingo game object.  Main data structure exists in the following form:
    
    Data objects 
    
    callnums (list):    numbers called in order
    bingoboards (list): list of arrays of bingo boards
    scoreboards (list): list of arrays of 0 filled boards to be marked with 1 
                        when same index bingo board wins 
    win_conds (list):   list of arrays with vertical and horizontal 1s indicating a 
                        winning score
    win_status (list):  list of booleans representing whether equivalent bingo boards 
                        have won already
    current_num_idx (int): index for current round and called number
    
    """
    
    def __init__(self, bingoboards:list, callnums:list):
        """
        Args:
        
        bingoboards (list): list of arrays representing bingo boards
        callnums (list):    list of called numbers in order of round
        """
        self.bingoboards = bingoboards
        self.callnums = callnums
        self.current_num_idx = 0 
        self.scoreboards = self.score_board_arrays(len(bingoboards))
        self.win_conds = self.win_condition_arrays()
        self.win_status = self.win_status_list(len(bingoboards))
        self.winning_board = None
        self.winning_marks = None
        self.winning_marked_board = None
        self.winning_num_idx = None
        self.winners = []
        
    def win_condition_arrays(self):
        empty = np.full((5,5),0)
        wins = []
        for i in range(empty.shape[0]):
            win = np.copy(empty)
            win[:,i] = 1
            wins.append(win)
        for i in range(empty.shape[1]):
            win = np.copy(empty)
            win[i,:] = 1
            wins.append(win)
        return wins
    
    def score_board_arrays(self, num:int):
        score_boards = []
        for i in range(num):
            score_boards.append(np.full((5,5),0))
        return score_boards
    
    def win_status_list(self, num:int):
        win_status = []
        for i in range(num):
            win_status.append(False)
        return win_status
    
    def next_num(self):
        if self.current_num_idx < len(self.callnums)-1:
            self.current_num_idx += 1
        
    def callnum_find(self):
        current_num = self.callnums[self.current_num_idx]
        for i, board in enumerate(self.bingoboards):
            found = np.where(board == current_num)
            if found[0].size > 0:
                self.callnum_mark_score(i,found[0][0],found[1][0])
            
    def callnum_mark_score(self, board_idx, mark_0, mark_1):
        if self.win_status[board_idx] == False:
            self.scoreboards[board_idx][mark_0][mark_1] = 1
        else:
            print('skipping, already won')
    
    def callnum_check_win(self):
        winners = []
        for i, score in enumerate(self.scoreboards):
            if self.win_status[i] == False:
                for win in self.win_conds:
                    cond = win*score
                    if (win*score==win).all():
                        self.win_status[i] = True
                        self.winning_num_idx = self.current_num_idx
                        winners.append(i)
        return winners               
            
    def find_all(self, mode='first'):
        for i in range(len(self.callnums)):
            print(f'calling number {self.callnums[i]}')
            self.callnum_find()
            board_nums = self.callnum_check_win()
            if len(board_nums) > 0:
                for board_num in board_nums:
                    print('\nWe have a winner!')
                    print(f'Winning board: \n{self.bingoboards[board_num]}')
                    print(f'Winning marks: \n{self.scoreboards[board_num]}')
                    self.winning_board = self.bingoboards[board_num]
                    self.marks = self.scoreboards[board_num]
                    self.winners.append((callnums[i], board_num, self.bingoboards[board_num],self.scoreboards[board_num]))
                    if mode == 'first':
                        break
                    elif mode == 'last':
                        pass
                    else:
                        raise ValueException('Mode must be "first" or "last"')
            self.next_num()
    
    def find_score(self):
        inverted = 1 - self.marks
        self.winning_marked_board = self.winning_board * inverted
        self.score = np.sum(self.winning_marked_board) * self.callnums[self.winning_num_idx]
        print(f'Marked board: \n{self.winning_marked_board}')
        print(f'Final score (unarmed sum) {np.sum(self.winning_marked_board)} * '\
              f'last called num {self.callnums[self.winning_num_idx]}')
        print(self.score)
        return self.score

In [504]:
# Workflow - part 1

# Input
callnums, bingoboards = bingo_input('data/d04_bingo.txt')

# Set up boards
bingo = Bingo(bingoboards, callnums)
# Go through call numbers, break at winning condition
bingo.find_all(mode='first')
# Determine winning score
bingo.find_score()

calling number 46
calling number 79
calling number 77
calling number 45
calling number 57
calling number 34
calling number 44
calling number 13
calling number 32
calling number 88
calling number 86
calling number 82
calling number 91
calling number 97
calling number 89
calling number 1
calling number 48
calling number 31
calling number 18
calling number 10
calling number 55
calling number 74
calling number 24
calling number 11
calling number 80
calling number 78
calling number 28
[60]

We have a winner!
Winning board: 
[[19 85 36 73 71]
 [65 62 14 52  3]
 [30 83 44 41  5]
 [55 15  0 61 95]
 [28 13 32 31 88]]
Winning marks: 
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [1 1 1 1 1]]
calling number 37
[51]

We have a winner!
Winning board: 
[[ 8 13 31 38 59]
 [86 94 55 10  1]
 [81 18 45 48 32]
 [43 25 37 49 67]
 [22 95 11 82 44]]
Winning marks: 
[[0 1 1 0 0]
 [1 0 1 1 1]
 [0 1 1 1 1]
 [0 0 1 0 0]
 [0 0 1 1 1]]
calling number 47
[68]

We have a winner!
Winning board: 
[[11 56 41  8

16168

In [505]:
# Part2 identical but switch find_all mode to set winning board to last rather than first

callnums, bingoboards = bingo_input('data/d04_bingo.txt')


#Set up boards
bingo = Bingo(bingoboards, callnums)
# Go through call numbers, break at winning condition
bingo.find_all(mode='last')
# Determine winning score
bingo.find_score()

calling number 46
calling number 79
calling number 77
calling number 45
calling number 57
calling number 34
calling number 44
calling number 13
calling number 32
calling number 88
calling number 86
calling number 82
calling number 91
calling number 97
calling number 89
calling number 1
calling number 48
calling number 31
calling number 18
calling number 10
calling number 55
calling number 74
calling number 24
calling number 11
calling number 80
calling number 78
calling number 28
[60]

We have a winner!
Winning board: 
[[19 85 36 73 71]
 [65 62 14 52  3]
 [30 83 44 41  5]
 [55 15  0 61 95]
 [28 13 32 31 88]]
Winning marks: 
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [1 1 1 1 1]]
calling number 37
[51]

We have a winner!
Winning board: 
[[ 8 13 31 38 59]
 [86 94 55 10  1]
 [81 18 45 48 32]
 [43 25 37 49 67]
 [22 95 11 82 44]]
Winning marks: 
[[0 1 1 0 0]
 [1 0 1 1 1]
 [0 1 1 1 1]
 [0 0 1 0 0]
 [0 0 1 1 1]]
calling number 47
[68]

We have a winner!
Winning board: 
[[11 56 41  8

[39, 50, 78]

We have a winner!
Winning board: 
[[21 56 79  8 54]
 [35 58 25 12 22]
 [27 75  7 93 86]
 [ 3 99  9 59 32]
 [45 40 64 60 68]]
Winning marks: 
[[1 1 1 0 1]
 [1 1 0 0 0]
 [0 0 1 0 1]
 [1 1 1 1 1]
 [1 0 1 0 1]]

We have a winner!
Winning board: 
[[19  3 87 72 90]
 [57 43 35  8 15]
 [51 10  4 42 56]
 [86 62 25 29 83]
 [97 49 28 70 59]]
Winning marks: 
[[0 1 0 0 0]
 [1 1 1 0 1]
 [1 1 0 1 1]
 [1 0 0 1 1]
 [1 1 1 1 1]]

We have a winner!
Winning board: 
[[12 23 17 46 91]
 [75 88 58 25  9]
 [65 64 62 49 33]
 [27 59 63 21 73]
 [56 31 87 81  0]]
Winning marks: 
[[0 1 1 1 1]
 [0 1 1 0 1]
 [1 1 0 1 0]
 [0 1 1 1 1]
 [1 1 0 1 0]]
calling number 40
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, already won
skipping, alrea

16168